In [1]:
import pandas as pd

In [6]:
consultas = pd.read_csv('/home/armando/git/insurance_case/datalake/raw/consultas_dummy.csv', sep=';')
print(consultas.shape)
display(consultas.head(3))

(350, 6)


,id,certificate_number,fecha_consulta,closure,specialty,placed_by
0,1,40,2024-08-21,"{""dayNote"": {""consultationObservation"": ""La pa...",general_medicine,286
1,2,87,2025-02-15,"{""dayNote"": {""consultationObservation"": ""✅ 78...",geriatrics,176
2,3,42,2025-06-14,"{""dayNote"": {""consultationObservation"": ""Ficha...",general_medicine,278


In [7]:
print(consultas.loc[0, 'closure'])

{"dayNote": {"consultationObservation": "La paciente se encuentra orientada en sus tres esferas y cooperadora, resto de exploración física diferida por video consulta.", "nextConsultationPending": "Mujer de la séptima década de la vida con los antecedentes ya mencionados, refiere adecuado seguimiento con médicos por antecedentes, se realiza estudios periódicamente. Durante la consulta se realizó llenado de historia clínica y se compartió información sobre vacuna contra neumococo. Se comparte también infografía que hacer en caso de emergencia. \nLa paciente refiere ocasionalmente dolores a nivel gastrointestinal así como articulares relacionados a fibromialgia, ya se descartaron causas orgánicas. Se sugiere realizar ESI el cual acepta. \nAgendar consulta posterior a ESI.", "needsPrescriptionOrMedicalOrder": "none"}, "patientGoal": "Iniciar seguimiento médico", "specialistGoal": "Llenado de HC, plan de seguimiento", "patientDiagnoses": [{"patientDiagnose": 4917}], "pauseConsultations": "

In [ ]:
import pandas as pd
import json

def get_field(x, field):

    result = None

    if field in ['consultationObservation', 'nextConsultationPending', 'needsPrescriptionOrMedicalOrder']:
        if not 'dayNote' in x.keys():
            return result
        
        if x['dayNote'] == []:
            return result

        if not field in x['dayNote'].keys():
            return result
            
        result = x['dayNote'][field]

    elif field in ['patientGoal', 'specialistGoal', 'pauseConsultations']:

        if not field in x.keys():
            return result
            
        result = x[field]

    return result

In [ ]:
    consultations = dbt.ref('consultas_dummy').to_pandas()

    #consultations = pd.read_csv('/home/armando/git/insurance_case/datalake/raw/consultas_dummy.csv', sep=';')
    consultations['closure'] = consultations['closure'].apply(lambda x: json.loads(x))
    consultations['day_note_consultation_observation'] = consultations['closure'].apply(lambda x: get_field(x, 'consultationObservation'))
    consultations['day_note_next_consultation_pending'] = consultations['closure'].apply(lambda x: get_field(x, 'nextConsultationPending'))
    consultations['day_note_needs_prescription_or_medical_order'] = consultations['closure'].apply(lambda x: get_field(x, 'needsPrescriptionOrMedicalOrder'))
    consultations['patiend_goal'] = consultations['closure'].apply(lambda x: get_field(x, 'patientGoal'))
    consultations['specialist_goal'] = consultations['closure'].apply(lambda x: get_field(x, 'specialistGoal'))
    consultations['pause_consultations'] = consultations['closure'].apply(lambda x: get_field(x, 'pauseConsultations'))

    consultations = consultations.drop(columns=['closure'])

    # Return the final Pandas DataFrame
    return consultations